In [100]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [101]:
# District Summary

# Create DafaFrames
school_df = pd.DataFrame(school_data)
student_df = pd.DataFrame(student_data)
complete_df = pd.DataFrame(school_data_complete)

# Number of schools
school_count = len(school_df["school_name"].unique())

# Total number of students
student_count = f'{school_df["size"].sum():,}'

# Total budget
total_budget = "${:,.2f}".format(school_df["budget"].sum())

# Average math score
avg_math_score = round(complete_df["math_score"].mean(), 6)

# Average reading score
avg_reading_score = round(complete_df["reading_score"].mean(), 6)

# % Passing Math
math_pass_count = 0
math_row_count = len(complete_df.math_score)
for row in complete_df["math_score"]:
    if row >= 70:
        math_pass_count = math_pass_count + 1
math_passing = round(math_pass_count / math_row_count * 100, 6)

# % Passing Reading
reading_pass_count = 0
reading_row_count = len(complete_df.reading_score)
for row in complete_df["reading_score"]:
    if row >= 70:
        reading_pass_count = reading_pass_count + 1
reading_passing = round(reading_pass_count / reading_row_count * 100, 6)
reading_passing

# Overall Passing (>=70 in both math and reading)
overall_passing = round(len(complete_df.query('math_score >= 70 & reading_score >= 70'))/ reading_row_count * 100, 6)

summary_df = pd.DataFrame({"Total Schools": school_count, "Total Students": student_count, "Total Budget": total_budget, 
              "Average Math Score":avg_math_score, "Average Reading Score": avg_reading_score, "% Passing Math": math_passing,
              "% Passing Reading": reading_passing, "% Overall Passing": overall_passing}, index=[0])
summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [102]:
# School Summary

# Shorten the 'complete_df', drop duplicates, set the index to the school, and sort based on the school name
short_df = complete_df[{"school_name", "type", "size", "budget", "math_score",
                        "reading_score"}].set_index("school_name").sort_values("school_name")
# Group the dataframe and get average scores
school_group_df = short_df.groupby(["school_name"])
mean_group_df = school_group_df.mean()

# Bin the scores and setup DataFrames for % Passing
bins = [0, 69, 100]
group_names = [0, 1]
math_df = short_df
math_df["math_percent"] = pd.cut(math_df["math_score"], bins, labels=group_names, include_lowest=True)
reading_df = short_df
reading_df["reading_percent"] = pd.cut(reading_df["reading_score"], bins, labels=group_names, include_lowest=True)
# If I display the short_df DataFrame here, the bins have been applied to it - I don't understand why this happens?

## Percentage of students passing math by school
# I don't quite understand the lambda function, the solution was taken from stackexchange.  It appears similar to a
# a list comprehension or 'If' conditional for the non-grouped columns and rows ?  Is there a better way to do this?
math_group = math_df.groupby("school_name")
math_group = math_group["math_percent"].value_counts(normalize=True).loc[lambda x : x >= .40].to_frame("% Passing Math")
# Merge # 1
merge_df = pd.merge(mean_group_df, math_group, how="left", on=["school_name"])

# Percentage of students passing reading by school
reading_group = reading_df.groupby("school_name")
reading_group = reading_group["reading_percent"].value_counts(normalize=True).loc[lambda x : x >= .40
                                                                                 ].to_frame("% Passing Reading")
# Merge # 2
merge_df = pd.merge(merge_df, reading_group, how="left", on=["school_name"])

## Overall Passing students by school
# Convert categorical data to numeric data
short_df["math_percent"]=pd.to_numeric(short_df.math_percent)
short_df["reading_percent"]=pd.to_numeric(short_df.reading_percent)
short_df["% Overall Passing"] = short_df["math_percent"] + short_df["reading_percent"]
overall_passing = short_df["% Overall Passing"].groupby(["school_name"]).value_counts(normalize=True
                                                                ).loc[lambda x : x >= .50].to_frame("% Overall Passing")
# Merge # 3
merge_df = pd.merge(merge_df, overall_passing, how="left", on=["school_name"])

# Calculate the "Per Student Budget"
merge_df["Per Student Budget"] = merge_df["budget"] / merge_df["size"]
# Add in District since it was dropped earlier due to being non-numeric during the grouping
merge_df = pd.merge(merge_df, school_data[["school_name","type"]], how="left", on=["school_name"])
# Rename the dataframe columns
renamed_df = merge_df.rename(columns={"school_name": "school_name", "type": "School Type", "size": "Total Students",
                                      "budget": "Total School Budget", "Per Student Budget": "Per Student Budget",
                                      "math_score": "Average Math Score", "reading_score": "Average Reading Score",
                                      "% Passing Math": "% Passing Math", "% Passing Reading": "% Passing Reading",
                                      "% Overall Passing": "% Overall Passing"})                                      
## Set the School Name to the Index and drop the column header (to match the answer result)
# apply the inplace argument to avoid creating another df
renamed_df.set_index(['school_name'], inplace=True)
renamed_df.index.name = None
renamed_df

# Reorder the dataframe to match the desired output
reorder_df = renamed_df[["School Type", "Total Students", "Total School Budget", "Per Student Budget",
                        "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading",
                        "% Overall Passing"]]

## Adjust the units to match the desired output
# drop the decimal from students
reorder_df["Total Students"] = reorder_df["Total Students"].astype(int)
# add thousands separator
reorder_df["Total Students"] = reorder_df["Total Students"].map("{:,}".format)
# print as currency
reorder_df["Total School Budget"] = reorder_df["Total School Budget"].map("${:,.2f}".format)
reorder_df["Per Student Budget"] = reorder_df["Per Student Budget"].map("${:,.2f}".format)
# convert to percent
reorder_df["% Passing Math"] = reorder_df["% Passing Math"] * 100
reorder_df["% Passing Reading"] = reorder_df["% Passing Reading"] * 100 # convert to percent
reorder_df["% Overall Passing"] = reorder_df["% Overall Passing"] * 100 # convert to percent

# The FINAL Product!
reorder_df
# The 'Total School Budget' and 'Per Student Budget' columns are combined into 'Total Student Budget' column in the 
# GitHub rendering.  Also, all the following columns are shifted to the left.  Funky.  Looks much cleaner in 
# Jupyter Notebook.

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [103]:
# Top Performing Schools (By % Overall Passing)
# Sort the reorder.df to show '% Overall Passing' in descending  order
top_performing_df = reorder_df.sort_values("% Overall Passing", ascending=False).head()
top_performing_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [104]:
# Bottom Performing Schools (By % Overall Passing)
# Sort the reorder.df to show '% Overall Passing' in ascending  order
bottom_performing_df = reorder_df.sort_values("% Overall Passing").head()
bottom_performing_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [116]:
## Math Scores by Grade
# Select the necessary columns
math_by_school = student_data[{"school_name", "grade", "math_score"}]
# Pivot the DataFrame
math_by_school = math_by_school.pivot_table(values="math_score", index="school_name", columns=["grade"])
# Re-order the columns
math_by_school = math_by_school[["9th", "10th", "11th", "12th"]]
# Drop the index name 'school'
math_by_school.index.name = None
# Drop the 'school' column name from the 1st column
# Solution modified from: https://stackoverflow.com/questions/33290374/pandas-pivot-table-column-names
# It joins the column name values from the math_by_school DataFrame to itself?
# Why is the statement in brackets?  Why does the statement start with ''?  Any insight?  Or another way to do this?
# Thanks in advance for any answers you have!
math_by_school.columns = [''.join(col) for col in math_by_school.columns.values]
math_by_school

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [118]:
## Reading Score by Grade
# Select the necessary columns
reading_by_school = student_data[{"school_name", "grade", "reading_score"}]
# Pivot the DataFrame
reading_by_school = reading_by_school.pivot_table(values="reading_score", index="school_name", columns=["grade"])
# Re-order the columns
reading_by_school = reading_by_school[["9th", "10th", "11th", "12th"]]
# Drop the index name 'school'
reading_by_school.index.name = None
# Drop the 'school' column name from the 1st column
reading_by_school.columns = [''.join(col) for col in reading_by_school.columns.values]
reading_by_school

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type